In [17]:
from itertools import combinations
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd


df = pd.read_csv('train_cleaned_small.csv')
df_val = pd.read_csv('val_cleaned_small.csv')

In [18]:
torch.manual_seed(0)

In [19]:
def clean_data(x):
    query_ids = sorted(list(set(x['query_id'])))
    ys_train = np.array(x[x['query_id']==query_ids[0]]['relevance_label'].tolist())
    ys_train_final = []
    ys_train_final.append(ys_train)
    for i in range(len(query_ids)):
        if i == 0:
            continue

        y_new = np.array(x[x['query_id']==query_ids[i]]['relevance_label'].tolist())
        ys_train_final.append(y_new)
    

    ys_train = torch.tensor(ys_train_final,dtype=torch.float32)
    
    X = np.array(x[x['query_id']==query_ids[0]].iloc[:,2:])
    x_train_final = []
    x_train_final.append(X)
    for i in range(len(query_ids)):
        if i == 0:
            continue

        x_new = np.array(x[x['query_id']==query_ids[i]].iloc[:,2:])
        x_train_final.append(x_new)
    

    X_train = torch.tensor(np.array(x_train_final),dtype=torch.float32)
    
    return X_train,ys_train
    

In [20]:
def dcg(scores):
    scores = np.array(scores,dtype = float)
    num = 2**scores-1
    for i in range(len(num)):
        num[i] /= np.log2(i+2)
    return np.sum(num)


def ndcg_k(scores, k):
    top_k = scores[:k]
    ideal_top_k = sorted(scores)[::-1][:k]
    ndcg = dcg(top_k)
    indcg = dcg(ideal_top_k)
    return ndcg/indcg

In [21]:
X_train,ys_train = clean_data(df)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

class ListNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_dim2, output_dim):
        super(ListNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim,hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

def KL_div(p, q):
    return (p * torch.log(p / q)).sum()


# Model hyperparameters
input_dim = 136
hidden_dim = 512
hidden_dim2 = 256
output_dim = 1
learning_rate = 0.01
num_epochs = 1

# Initialize model and optimizer
model = ListNet(input_dim, hidden_dim,hidden_dim2, output_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train model
for q in range(X_train.shape[0]):
    X = X_train[q]
    Y = torch.tensor(ys_train[q].reshape(50),dtype = torch.float64)
    for epoch in range(num_epochs):
#         model.train()
        optimizer.zero_grad()
        output = model(X)
        output = nn.functional.softmax(output, dim=0)
        Y = Y.reshape(50,1)
        target = nn.functional.softmax(Y,dim=0)
        loss = KL_div(output, target).float()
        loss.backward()
        optimizer.step()
        print("Epoch: {}, Loss: {:.4f}".format(epoch+1, loss.item()))


C:\Users\chana\AppData\Local\Temp\ipykernel_24080\867369768.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(ys_train[q].reshape(50),dtype = torch.float64)


Epoch: 1, Loss: 0.4288
Epoch: 1, Loss: 0.4346
Epoch: 1, Loss: 1.1254
Epoch: 1, Loss: 0.2305
Epoch: 1, Loss: 0.3009
Epoch: 1, Loss: 0.3652
Epoch: 1, Loss: 0.2998
Epoch: 1, Loss: 0.2421
Epoch: 1, Loss: 0.5149
Epoch: 1, Loss: 0.3993
Epoch: 1, Loss: 0.3688
Epoch: 1, Loss: 0.3813
Epoch: 1, Loss: 0.3102
Epoch: 1, Loss: 0.2089
Epoch: 1, Loss: 0.3035
Epoch: 1, Loss: 0.4956
Epoch: 1, Loss: 0.3158
Epoch: 1, Loss: 0.3562
Epoch: 1, Loss: 0.5776
Epoch: 1, Loss: 0.3948
Epoch: 1, Loss: 0.2637
Epoch: 1, Loss: 0.1746
Epoch: 1, Loss: 0.2860
Epoch: 1, Loss: 0.3851
Epoch: 1, Loss: 0.2539
Epoch: 1, Loss: 0.2975
Epoch: 1, Loss: 0.2415
Epoch: 1, Loss: 0.3761
Epoch: 1, Loss: 0.3335
Epoch: 1, Loss: 0.6082
Epoch: 1, Loss: 0.1871
Epoch: 1, Loss: 0.8572
Epoch: 1, Loss: 0.6419
Epoch: 1, Loss: 0.1619
Epoch: 1, Loss: 0.4678
Epoch: 1, Loss: 0.2719
Epoch: 1, Loss: 0.1566
Epoch: 1, Loss: 0.1466
Epoch: 1, Loss: 0.2197
Epoch: 1, Loss: 0.3855
Epoch: 1, Loss: 0.2217
Epoch: 1, Loss: 0.3209
Epoch: 1, Loss: 0.2209
Epoch: 1, L

In [23]:
n = X_train.shape[0]
ndcg_list = []
max_ndcg = 0
for i in range(n):
    output = model(X_train[i])
    output = nn.functional.softmax(output,dim = 0)
    output = np.array(output.detach().numpy())
    output = output.reshape(50)

    Y = np.array(ys_train[i].reshape(50))
    
    rank_pred = np.argsort(output)[::-1]
    rank_score = Y[rank_pred]
    ndcg = ndcg_k(rank_score, 10)
    ndcg_list.append(ndcg)
    if ndcg > max_ndcg:
        max_ndcg = ndcg
    

C:\Users\chana\AppData\Local\Temp\ipykernel_24080\2468370219.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return ndcg/indcg


In [24]:
np.nanmean(ndcg_list)

0.4655806602111526

In [25]:
idx = 181
output = model(X_train[idx])
output = nn.functional.softmax(output,dim = 0)
output = np.array(output.detach().numpy())
output = output.reshape(50)

Y = np.array(ys_train[idx].reshape(50))

In [26]:
rank_pred = np.argsort(output)[::-1]
rank_score = Y[rank_pred]

In [29]:
ndcg_k(rank_score,10)

0.9740816258588203

In [30]:
X_val, ys_val = clean_data(df_val)

In [31]:
n = X_val.shape[0]
ndcg_list = []
max_ndcg = 0
for i in range(n):
    output = model(X_val[i])
    output = nn.functional.softmax(output,dim = 0)
    output = np.array(output.detach().numpy())
    output = output.reshape(50)

    Y = np.array(ys_val[i].reshape(50))
    
    rank_pred = np.argsort(output)[::-1]
    rank_score = Y[rank_pred]
    ndcg_val = ndcg_k(rank_score, 10)
    ndcg_list.append(ndcg_val)
    if ndcg_val > max_ndcg:
        max_ndcg = ndcg_val
    

0
1
3
5
10
16
23
65


In [32]:
np.nanmean(ndcg_list)

0.4316787742719746